In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers
import joblib

# อ่านข้อมูล
data = pd.read_csv('/content/accident.csv')  # แทนที่ด้วยชื่อไฟล์จริงถ้ามี

# จัดการ missing data
# เติม Speed_of_Impact ด้วยค่า mean
data['Speed_of_Impact'] = data['Speed_of_Impact'].fillna(data['Speed_of_Impact'].mean())
# เติม Age ด้วยค่า mean (ถ้ามี missing)
data['Age'] = data['Age'].fillna(data['Age'].mean())
# เติม Gender ด้วย 'Unknown' (ถ้ามี missing)
data['Gender'] = data['Gender'].fillna('Unknown')

# แปลง categorical variables เป็น numerical
le = LabelEncoder()
data['Seatbelt_Used'] = le.fit_transform(data['Seatbelt_Used'])
data['Helmet_Used'] = le.fit_transform(data['Helmet_Used'])
data['Gender'] = le.fit_transform(data['Gender'])  # แปลง Gender (Male/Female/Unknown) เป็นตัวเลข

# ตรวจสอบข้อมูลหลังจัดการ missing values
print("Missing values after imputation:")
print(data.isnull().sum())

# เตรียม features และ target (เพิ่ม Age และ Gender)
X = data[['Seatbelt_Used', 'Speed_of_Impact', 'Helmet_Used', 'Age', 'Gender']].values
y = data['Survived'].values

# แปลง output: 1 (รอด) -> 0 และ 0 (ไม่รอด) -> 1
y = 1 - y

# แบ่ง train/test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# สร้างโมเดล (รับ 5 inputs)
model = keras.Sequential([
    layers.Input(shape=(5,)),  # ปรับให้รับ 5 inputs
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile โมเดล
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train โมเดล
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=1,
    validation_split=0.2,
    verbose=1
)

# Evaluate โมเดล
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"\nTest accuracy: {test_accuracy:.4f}")

# เซฟโมเดล
# วิธีที่ 1: เซฟเป็น .pkl ด้วย joblib
joblib.dump(model, 'neural_network_model.pkl')
print("Model saved as neural_network_model.pkl using joblib")

# วิธีที่ 2: เซฟแบบ Keras native (แนะนำ)
model.save('neural_network_model.keras')
print("Model saved as neural_network_model.keras using Keras native format")

# ตัวอย่างการโหลดและใช้งาน
loaded_model_keras = keras.models.load_model('neural_network_model.keras')

# ทดสอบ predict (ปรับให้รับ 5 inputs)
sample = np.array([[1, 50, 1, 30, 1]])  # Seatbelt_Yes, Speed_50, Helmet_Yes, Age_30, Gender_Male(1)
prediction = loaded_model_keras.predict(sample)
print(f"Prediction (0 = survived, 1 = not survived): {prediction[0][0]:.4f}")

Missing values after imputation:
Age                0
Gender             0
Speed_of_Impact    0
Helmet_Used        0
Seatbelt_Used      0
Survived           0
dtype: int64
Epoch 1/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5736 - loss: 9.3653 - val_accuracy: 0.4375 - val_loss: 0.8745
Epoch 2/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4339 - loss: 0.8838 - val_accuracy: 0.4375 - val_loss: 0.7413
Epoch 3/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5462 - loss: 0.8422 - val_accuracy: 0.5938 - val_loss: 0.9366
Epoch 4/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4598 - loss: 1.0441 - val_accuracy: 0.5938 - val_loss: 0.7129
Epoch 5/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4043 - loss: 0.8491 - val_accuracy: 0.6250 - val_loss: 0.6671
Epoch 6/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4764 - loss: 0.7311 - val_accuracy: 0.5938 - val_loss: 0.7822
Epoch 7/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step -